In [1]:
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
from keras.datasets import imdb

Using TensorFlow backend.


In [2]:
n_words = 1000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=n_words)
print('Train seq: {}'.format(len(X_train)))
print('Test seq: {}'.format(len(X_train)))

Train seq: 25000
Test seq: 25000


In [3]:
print('Train example: \n{}'.format(X_train[0]))
print('\nTest example: \n{}'.format(X_test[0]))

# Note: the data is already preprocessed (words are mapped to vectors)

Train example: 
[1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]

Test example: 
[1, 89, 27, 2, 2, 17, 199, 132, 5, 2, 16, 2, 24, 8, 760, 4, 2, 7, 4, 22, 2, 2, 16, 2, 17, 2, 7, 2

In [4]:
# Pad sequences with max_len
max_len = 200
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

In [5]:
# Define network architecture and compile
model = Sequential()
model.add(Embedding(n_words, 50, input_length=max_len))
model.add(Dropout(0.5))
model.add(Conv1D(128, 3, padding='valid', activation='relu', strides=1,))
model.add(GlobalMaxPooling1D())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',  optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 50)           50000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 198, 128)          19328     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               32250     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total para

In [6]:
callbacks = [EarlyStopping(monitor='val_acc', patience=3)]

In [7]:
batch_size = 64
n_epochs = 100

model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs, validation_split=0.2, callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/100
20000/20000 [==============================] - 4s - loss: 0.5609 - acc: 0.6845 - val_loss: 0.4027 - val_acc: 0.8196
Epoch 2/100
20000/20000 [==============================] - 2s - loss: 0.3884 - acc: 0.8278 - val_loss: 0.3528 - val_acc: 0.8488
Epoch 3/100
20000/20000 [==============================] - 2s - loss: 0.3499 - acc: 0.8472 - val_loss: 0.3503 - val_acc: 0.8430
Epoch 4/100
20000/20000 [==============================] - 2s - loss: 0.3275 - acc: 0.8617 - val_loss: 0.3173 - val_acc: 0.8648
Epoch 5/100
20000/20000 [==============================] - 2s - loss: 0.3144 - acc: 0.8659 - val_loss: 0.3168 - val_acc: 0.8606
Epoch 6/100
20000/20000 [==============================] - 2s - loss: 0.2990 - acc: 0.8728 - val_loss: 0.3103 - val_acc: 0.8672
Epoch 7/100
20000/20000 [==============================] - 2s - loss: 0.2967 - acc: 0.8756 - val_loss: 0.3084 - val_acc: 0.8688
Epoch 8/100
20000/20000 [==============================

In [8]:
print('\nAccuracy on test set: {}'.format(model.evaluate(X_test, y_test)[1]))

# Accuracy on test set: 0.873

24448/25000 [============================>.] - ETA: 0s
Accuracy on test set: 0.873
